In [2]:
using DataFrames
using CSV
using Statistics
using ScikitLearn

In [26]:
@sk_import linear_model: LinearRegression
@sk_import linear_model: ElasticNet

PyObject <class 'sklearn.linear_model.coordinate_descent.ElasticNet'>

In [12]:
KEY = "2018chs"
COLUMN = :totalPoints

:totalPoints

In [13]:
headers = CSV.read("../data/2018_headers.csv") |> (h -> String.(names(h))) |> (a -> push!(a, "key"))
scores = CSV.read("../data/matches_$KEY.csv", header=headers)
teams = CSV.read("../data/matches_teams_$KEY.csv", header=["team1","team2","team3","key"])

teamnums = teams.team1 |> unique |> sort
teamindex = Dict(t => i for (i, t) in enumerate(teamnums))

ß = zeros(size(scores,1), size(teamnums,1))
for p in [:team1, :team2, :team3]
    for i=1:size(teams,1)
        team = teams[i,p]
        j = teamindex[team]
        ß[i,j] = 1.0
    end
end
y = convert.(Float64, coalesce(scores[COLUMN], -1.0))

1510-element Array{Float64,1}:
 436.0
 221.0
 423.0
 202.0
 335.0
 342.0
 275.0
 411.0
 466.0
 305.0
 371.0
 218.0
 173.0
   ⋮  
 194.0
 246.0
 407.0
 260.0
 307.0
 315.0
 276.0
 361.0
 381.0
 218.0
 333.0
 258.0

In [16]:
model = ScikitLearn.fit!(LinearRegression(fit_intercept=false), ß, y)

PyObject LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [32]:
oprs = DataFrame(hcat(teamnums, model[:coef_]), [:team, :rating])
sort(oprs, (:rating), rev = true)

,team,rating
,Any,Any
1,frc225,180.061
2,frc1629,179.407
3,frc1731,161.824
4,frc1262,151.46
5,frc623,145.201
6,frc384,144.733
7,frc4472,143.854
8,frc5945,142.947
9,frc5546,142.848


In [35]:
model2 = ScikitLearn.fit!(ElasticNet(alpha=0.25, fit_intercept=false), ß, y)
oprs2 = DataFrame(hcat(teamnums, model2[:coef_]), [:team, :rating])
sort(oprs2, (:rating), rev = true)

,team,rating
,Any,Any
1,frc1629,67.4714
2,frc384,59.7123
3,frc1731,58.9891
4,frc1262,58.3808
5,frc1885,56.545
6,frc5546,56.5257
7,frc836,55.3608
8,frc4472,54.5846
9,frc612,54.3343
